<a href="https://colab.research.google.com/github/mbc2009/Lmp_ML/blob/main/Model_trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Enviornment Initialization

In [1]:
%%capture
%%bash

# remove unnecessary
rm -rf *

# update pip
python -m pip install --upgrade pip

# install package
pip install opencv-python pillow
pip install segmentation_models_pytorch
pip install -q kaggle
pip install dropbox
pip install scikit-image

In [2]:
# basic import
import  os, sys, time, math, random, math, psutil, h5py
from    typing                  import  List, Tuple
from    dropbox                 import  Dropbox
from    tqdm                    import  tqdm
from    matplotlib              import  pyplot      as plt
import  numpy                                       as np
import  pandas                                      as pd
import  zipfile
import  warnings
import  shutil

from    skimage                 import  io
from    scipy                   import  interpolate
from    scipy.interpolate       import  RegularGridInterpolator
from    scipy.ndimage           import  generic_filter
from    PIL                     import  Image

import  torch
from    torch                   import  nn
from    torch.nn                import  functional  as F
import  torch.optim                                 as optim
import  torchvision.transforms.functional           as TF
from    torch.utils.data        import  Dataset, DataLoader, TensorDataset, random_split, Subset
from    torchvision             import  transforms, models
from    torchvision.transforms  import  *
import  kagglehub

In [3]:
# check hardware
print(f"CPU core #:\t{os.cpu_count()}")
print(f"CPU threads #:\t{psutil.cpu_count(logical=True)}")
print(f"Total memory:\t\t{psutil.virtual_memory().total / (1024**3):.2f} GB")
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"available GPU #:\t{gpu_count}")
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i+1}:\t\t{gpu_name}")
else:
    print("No available GPU")

CPU core #:	2
CPU threads #:	2
Total memory:		12.67 GB
No available GPU


# 1. Configuration

In [14]:
class Confuration():
  # define paths to directory
  home_dir                = os.path.expanduser("~")
  working_dir             = os.path.join(os.getcwd(),'Lmp_ML')
  DataBase_dir            = os.path.join(working_dir,'DataBase')
  DB_3D_Grids_path        = os.path.join(DataBase_dir, '4', '3D_Grids.h5') # TODO: choose database version, here ver=4

conf = Confuration()

In [5]:
# import code from git hub
!git clone https://github.com/mbc2009/Lmp_ML

Cloning into 'Lmp_ML'...
remote: Enumerating objects: 511, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 511 (delta 21), reused 11 (delta 4), pack-reused 450 (from 3)
Receiving objects: 100% (511/511), 128.60 MiB | 9.28 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [6]:
# make directory
os.makedirs(conf.DataBase_dir, exist_ok=True)

In [7]:
# download
original_path = kagglehub.dataset_download("mbc2009/heat-and-mass-transfer",force_download=True)
shutil.move(original_path, conf.DataBase_dir)

100%|██████████| 773M/773M [00:19<00:00, 41.4MB/s]

Extracting files...


'/content/Lmp_ML/DataBase/4'

# 2. Functions and Classes

In [ ]:
def check_hdf5_content(file_path:str,PrintStrcut=False)->int:
  '''
  check the content (name, quantity) of hdf5 file
  input:
    file_path: the path of hdf5 file
    PrintStrcut: print the structure of hdf5 file
  return:
    the name and quantity of variables in hdf5 file
  '''
  # 初始化数据集计数器
  dataset_count = 0

  # 定义一个内部函数用于遍历 HDF5 文件内部
  def count_datasets(name, obj):
        nonlocal dataset_count
        if isinstance(obj, h5py.Dataset):  # 判断是否为数据集
            dataset_count += 1
        elif isinstance(obj, h5py.Group):  # 判断是否为组
            pass  # 如果是组，不计数

  with h5py.File(file_path, "r") as h5f:

        # 遍历文件内容以计数
        h5f.visititems(count_datasets)

        # 打印所有内变量名字
        if PrintStrcut:
          print(f"文件结构:")
          h5f.visit(print)

  print(f'文件数: {dataset_count}')

  return dataset_count # 文件数


# 3.Prepare Data Base

## 3.1.Check data base

In [18]:
num_grids = check_hdf5_content(conf.DB_3D_Grids_path)

文件数: 836


In [23]:
DB_3D_Grids = h5py.File(conf.DB_3D_Grids_path, "r")
GA = DB_3D_Grids[f"len_{2}_sigma_{8}_{343}"][:]
print(f'矩阵形状: {GA.shape}')

矩阵形状: (400, 400, 400)
